In [61]:

import GPSPreprocess as gpsp
import pytest
import geopandas as gpd
import pandas as pd
import os
import sys
import re
import datetime

sample_gps_file_path = os.getcwd().split(
"PyERT-BLACK")[0] + 'PyERT-BLACK/quarto-example/data/sample-gps'

sample_gps = f'{sample_gps_file_path}/sample-gps-3.csv'
Data = pd.read_csv(sample_gps)
DataLength = len(Data)
GPSData = gpsp.GPSPreprocess(data=Data)

ProcessedData = GPSData.getData()
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     print(ProcessedData.head(1000))


        
Processed = ProcessedData.reset_index()
stage_points = {}
pattern = '(\d+)\/(\d+)\/(\d+)\s(\d+):(\d+)'
prev_time = 0
episode_len = 0
temp_episode_len = 0
start_index = 0
ep_id= 0
valid_stop = False
last_car_index = -1
last = len(ProcessedData.index) - 1
for index,row in ProcessedData.iterrows():
    speed = float(row['Speed_kmh'])
    #print(speed)
    
    
    if (speed <= 10.008):
        new_mode = 'walk/stop' 
        if (speed > 0.36): 
            valid_stop = False
    elif (speed > 10.008):
        new_mode = 'car'

    pattern = '(\d+)\/(\d+)\/(\d+)\s(\d\d):(\d\d)'
    match = re.search(pattern, row['LocalTime'])
    cur_time = datetime.datetime(year = int(match.groups()[2]),month = int(match.groups()[1]), day = int(match.groups()[0]), hour = int(match.groups()[3]), minute = int(match.groups()[4]))
    if index == 0:
        cur_mode = new_mode
        prev_time = cur_time
    time_difference = (cur_time - prev_time).total_seconds()
    prev_time = cur_time
    episode_len += time_difference
    
    
    if (time_difference > 120): 
        #print(f'GAP time_difference: {episode_len} cur_mode: {cur_mode}')
        ep_id+=1
        stage_points[ep_id] = {}
        stage_points[ep_id]['start'] = start_index
        stage_points[ep_id]['end'] = index
        if cur_mode == 'walk/stop' and valid_stop == True:
            stage_points[ep_id]['mode'] = 'stop'
        elif cur_mode == 'walk/stop' and valid_stop == False:
            stage_points[ep_id]['mode'] = 'walk'
        else:
            stage_points[ep_id]['mode'] = 'car'

        episode_len = 0
        start_index = index + 1
        cur_mode = new_mode
    elif (index == last):
        if (cur_mode == 'car' and episode_len > 120):
            ep_id +=1
            stage_points[ep_id] = {}
            stage_points[ep_id]['start'] = start_index
            stage_points[ep_id]['end'] = index
            stage_points[ep_id]['mode'] = 'car'
        elif (cur_mode == 'walk/stop' and episode_len > 60 and valid_stop == False):
            ep_id +=1
            stage_points[ep_id] = {}
            stage_points[ep_id]['start'] = start_index
            stage_points[ep_id]['end'] = index
            stage_points[ep_id]['mode'] = 'walk'
        elif (cur_mode == 'walk/stop' and episode_len > 60 and valid_stop == True):
            ep_id +=1
            stage_points[ep_id] = {}
            stage_points[ep_id]['start'] = start_index
            stage_points[ep_id]['end'] = index
            stage_points[ep_id]['mode'] = 'stop'
    else:
        if (new_mode != cur_mode):
            if (new_mode == 'walk/stop') and cur_mode == 'car': #currently, no invalid car episodes
                if (last_car_index < 0):
                    last_car_index = index - 1
                    temp_episode_len = 0

                temp_episode_len += time_difference
                #print('temp_episode_len: ' + str(temp_episode_len))
                if ( temp_episode_len > 60 ):
                    ep_id +=1
                    stage_points[ep_id] = {}
                    stage_points[ep_id]['start'] = start_index
                    stage_points[ep_id]['end'] = last_car_index
                    stage_points[ep_id]['mode'] = 'car'
                    start_index = last_car_index + 1
                    last_car_index = -1
                    episode_len = temp_episode_len
                    cur_mode = new_mode
            elif new_mode == "car" and (cur_mode == 'walk/stop') :  
                if ((episode_len > 60 and valid_stop)): #valid stop
                    #print(f'valid episode ep_len: {episode_len} cur_mode: {cur_mode} new_mode: {new_mode}')
                    ep_id +=1
                    stage_points[ep_id] = {}
                    stage_points[ep_id]['start'] = start_index
                    stage_points[ep_id]['end'] = index - 1
                    stage_points[ep_id]['mode'] = 'stop'
                elif ((episode_len > 60 and not valid_stop)): #invalid stop, valid walk
                    #print(f'valid episode ep_len: {episode_len} cur_mode: {cur_mode} new_mode: {new_mode}')
                    ep_id +=1
                    stage_points[ep_id] = {}
                    stage_points[ep_id]['start'] = start_index
                    stage_points[ep_id]['end'] = index - 1
                    stage_points[ep_id]['mode'] = 'walk'
                else :
                    #print(f'invalid episode')
                    if (ep_id == 0):
                        stage_points[ep_id] = {}
                        stage_points[ep_id]['start'] = start_index
                        stage_points[ep_id]['end'] = index - 1
                        stage_points[ep_id]['mode'] = 'invalid'
                    if (ep_id != 0):
                        stage_points[ep_id]['end'] = index - 1
                
                start_index = index
                episode_len = 0
                cur_mode = new_mode
                valid_stop = True
        else:
            if temp_episode_len > 0:
                last_car_index = -1
                temp_episode_len = 0

serial_ids = []
record_ids = []
start_times = []
modes = []
geometry = []

invalid_first_stage = False

for key in stage_points:
    print(f"key: {key} start: {stage_points[key]['start']} end: {stage_points[key]['end']} mode: {stage_points[key]['mode']}")
    if stage_points[key]['mode'] == 'invalid':
        invalid_first_stage = True
        target = ProcessedData.iloc[stage_points[key]['start']]
    else:
        if (not invalid_first_stage):
            target = ProcessedData.iloc[stage_points[key]['start']]
        serial_ids.append(target.SerialID)
        record_ids.append(target.RecordID)
        start_times.append(target.LocalTime)
        modes.append(stage_points[key]['mode'])
        geometry.append(target.geometry)

data = { 'EPISODEID' : serial_ids, 
        'RECORDID' : record_ids, 
        'TIMESTART' : start_times, 
        'MODES' : modes,
        'geometry': geometry}
gdf = gpd.GeoDataFrame(data)
print(gdf)



AttributeError: 'NoneType' object has no attribute 'groups'